In [1]:
%matplotlib inline
import hashlib
import string
import matplotlib.pyplot as plt
import numpy as np


A = string.ascii_letters
M = 26
Z = 20
N = 100

Key functions

In [2]:
def gen_seed(x_k, x_l):
    str_repr = ''.join(x_k.astype(str)) + ''.join(x_l.astype(str))
    return int(hashlib.sha256(str_repr.encode('utf-8')).hexdigest(), 16)  % 10**9
    

def braid(x_k, x_l, q):
    np.random.seed(gen_seed(x_k, x_l))
    mask = np.random.rand(len(x_k)) < 1/(q+1)
    u = x_k.copy()
    u[mask] = x_l[mask]
    return u


def hamming(x_k, x_l):
    return np.mean(x_k != x_l)

Setup

In [3]:
np.random.seed(1)
x_star = np.random.randint(0, Z, N)
xs = [np.random.randint(0, Z, N) for i in range(M)]

def encode(s):
    y = x_star.copy()
    for i, a in enumerate(s):
        idx = A.index(a)
        y = braid(y, xs[idx], i+1)
    return y


def decode(y, l=None):
    # get *set* of elements in sequence
    min_idxs = np.argsort([hamming(x, y) for x in xs])[:l]
    vs = [xs[idx] for idx in min_idxs]
    # reconstruct sequence
    y_star = x_star.copy()
    s = ''
    for i in range(1, l+1):
        us = [braid(y_star, v, i) for v in vs]
        j = np.argmin([hamming(u, y) for u in us])
        s += A[min_idxs[j]]
        y_star = braid(y_star, vs[j], i)
    return s

Demo

In [4]:
print('Hamming similarities between symbols:')
symbol_dists = []
for k in range(M-1):
    for l in range(k+1, M):
        symbol_dists.append(1 - hamming(xs[k], xs[l]))
print('Min = ', np.min(symbol_dists))
print('Max = ', np.max(symbol_dists))
print('Mean = ', np.mean(symbol_dists))
print('Std = ', np.std(symbol_dists))

Hamming similarities between symbols:
Min =  0.0
Max =  0.13
Mean =  0.049507692307692315
Std =  0.023371332259792817


In [5]:
test_seqs = [
    'abcde',
    'abced',
    'aabcd',
    'zyxwv',
    'zyxvw',
    'aaazz',
]

for s in test_seqs:
    print(s, '--> y --> ', decode(encode(s), 5))

abcde --> y -->  abcde
abced --> y -->  abced
aabcd --> y -->  aabcd
zyxwv --> y -->  zyxwv
zyxvw --> y -->  zyxvw
aaazz --> y -->  aaazz
